# Spike Train Calculation

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
import neuron.spikes

In [7]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Functions

In [76]:
def filter_spike_times(arr_2d, start, stop, padding=-1):
    """
    Filters a 2D array based on a start and stop condition and pads shorter arrays to match the length of the longest array.

    Parameters:
    arr_2d (numpy.ndarray): 2D array to be filtered and padded.
    start (int or float): Lower bound of the condition.
    stop (int or float): Upper bound of the condition.
    padding (int or float): Value to use for padding, defaults to -1.

    Returns:
    numpy.ndarray: A 2D array where each inner array has been filtered based on the condition and padded to match the length of the longest array.
    """

    # Filter each row in arr_2d based on the condition
    masked_data = [row[(row >= start) & (row < stop)] for row in arr_2d]

    # Determine the maximum length of arrays in the list
    max_length = max(len(arr) for arr in masked_data)

    # Pad each array in masked_data with the padding value so they all have the same length (max_length)
    padded_arrays = [np.concatenate([x, np.full([max_length - len(x)], padding)]) for x in masked_data]

    # Convert the list of arrays to a 2D numpy array
    padded_arrays = np.array(padded_arrays)

    return padded_arrays

## Inputs & Data

Explanation of each input and where it comes from.

In [8]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

LFP_SPECTRAL_DF = pd.read_pickle("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention/results/2024_01_30_rce2_spectral_preprocessing/proc/rce_pilot_2_04_spectral_and_sleap.pkl")

# ALL_PHY_DIR = glob.glob("/scratch/back_up/reward_competition_extention/final_proc/phy_curation/*")
ALL_PHY_DIR = glob.glob("/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/*")


OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

SAMPLING_RATE = 20000
SPIKE_WINDOW = 2000

In [9]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,reward_port,box_bottom_left,box_bottom_right,agent,subject_locations,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
0,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1.1,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
1,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 930], [930, 948...","[[32974, 32976], [33201, 33207], [33208, 33211...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1.1,...,"[48.01956210909252, 37.264554576109816]","[33.1534452699471, 35.61101869926864]","[62.60288905131385, 35.71974322144712]",None,"[[[27.67316905433137, 31.06298795228111], [27....",NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,"[21.36, 21.36, 21.36, 21.36, 21.36, 21.36, 21....",NaN
2,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,1.2,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",None,"[[[34.36404036584844, 35.615553316175784], [34...",NaN,"[55.4, 52.03, 48.72, 45.53, 42.44, 39.44, 36.5...",NaN,"[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,...",NaN
3,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,1.2,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.1,"[[[34.36404036584844, 35.615553316175784], [34...","[[[46.68890098287189, 11.495085446056166], [43...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1..."
4,2,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...","[[33601, 33798], [34108, 34165], [34166, 34179...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.1,1.1,...,"[45.31189572048577, 36.38317068198775]","[30.490085836306232, 34.91627159792731]","[60.10744182471764, 34.262899786598894]",None,"[[[46.71964110607613, 36.493379374373234], [44...",NaN,"[5.8, 5.207, 4.64, 4.1, 3.596, 3.127, 2.7, 2.3...",NaN,"[4.633, 4.09, 4.36, 4.656, 4.65, 4.383, 4.35, ...",NaN


## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [10]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [11]:
FULL_LFP_TRACES_PKL = "{}_05_lfp_spectral_sleap_spikes.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

# Reading in Phy

- Reading in a spreadsheet of all the unit classifications
    - They are divided up into good units, multi-units, and noise

In [12]:
ALL_PHY_DIR

['/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230617_115521_standard_comp_to_omi

In [13]:
recording_to_cluster_info = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        recording_to_cluster_info[recording_basename] = pd.read_csv(file_path, sep="\t")
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/cluster_info.tsv'


In [14]:
recording_to_cluster_info[list(recording_to_cluster_info.keys())[1]].head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,3,71,22,243,8.729426,1,0,20.0,9.474292,mua,33069,0,5
1,5,4,0,14,9.769127,2,0,40.0,1.879731,mua,6561,0,7
2,6,2,0,11,13.532221,2,0,40.0,11.421351,good,39865,0,8
3,7,58,30,99,9.302568,2,0,40.0,20.851808,good,72781,0,9
4,8,29,12,94,7.241092,3,0,60.0,4.613521,mua,16103,0,10


- Combining all the unit info dataframes and adding the recording name

In [15]:
recording_to_cluster_info_df = pd.concat(recording_to_cluster_info, names=['recording_name']).reset_index(level=1, drop=True).reset_index()


In [16]:
recording_to_cluster_info_df.head()

,recording_name,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,20230620_114347_standard_comp_to_omission_D4_s...,0,12,9,45,10.498872,0,0.0,0.0,4.379580,mua,14970,0,1.0
1,20230620_114347_standard_comp_to_omission_D4_s...,11,0,0,0,10.311028,5,0.0,100.0,0.230828,mua,789,0,12.0
2,20230620_114347_standard_comp_to_omission_D4_s...,14,4,0,7,8.454960,8,0.0,160.0,1.908058,good,6522,0,15.0
3,20230620_114347_standard_comp_to_omission_D4_s...,21,148,57,355,9.774931,11,0.0,220.0,10.004284,mua,34196,0,23.0
4,20230620_114347_standard_comp_to_omission_D4_s...,22,105,26,351,8.435524,12,0.0,240.0,13.887395,mua,47469,0,24.0


- Filtering for the good units

In [17]:
good_unit_cluster_info_df = recording_to_cluster_info_df[recording_to_cluster_info_df["group"] == "good"].reset_index(drop=True)

In [18]:
good_unit_cluster_info_df.head()

,recording_name,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,20230620_114347_standard_comp_to_omission_D4_s...,14,4,0,7,8.454960,8,0.0,160.0,1.908058,good,6522,0,15.0
1,20230620_114347_standard_comp_to_omission_D4_s...,28,2,1,7,8.708598,14,0.0,280.0,1.436163,good,4909,0,30.0
2,20230620_114347_standard_comp_to_omission_D4_s...,41,2,1,3,10.409801,19,0.0,380.0,1.583319,good,5412,0,43.0
3,20230620_114347_standard_comp_to_omission_D4_s...,42,6,1,8,11.904501,20,0.0,400.0,12.966718,good,44322,0,44.0
4,20230620_114347_standard_comp_to_omission_D4_s...,43,1,0,4,7.047093,26,0.0,520.0,1.489701,good,5092,0,45.0


In [19]:
recording_to_good_unit_ids = good_unit_cluster_info_df.groupby('recording_name')['cluster_id'].apply(list).to_dict()

- A list of all the unit IDs that each spike came from in order
    - First item is first spike, second item is second spike, etc.

In [20]:
recording_to_spike_clusters = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        recording_to_spike_clusters[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/spike_clusters.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/spike_clusters.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/spike_clusters.npy'


In [21]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]]

array([ 42,  42, 207, ...,  22, 229,  68], dtype=int32)

In [22]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]].shape

(903326,)

- The times that all the spikes happened

In [23]:
recording_to_spike_times = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        recording_to_spike_times[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/spike_times.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/spike_times.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/spike_times.npy'


In [24]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]]

array([[     120],
       [     262],
       [     439],
       ...,
       [68362487],
       [68362524],
       [68362610]])

In [25]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]].shape

(903326, 1)

### Combining everything into a dataframe

In [26]:
recording_to_spike_df = {}

for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        cluster_info_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        cluster_info_df = pd.read_csv(cluster_info_path, sep="\t")

        spike_clusters_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        spike_clusters = np.load(spike_clusters_path)
        
        spike_times_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        spike_times = np.load(spike_times_path)

        spike_df = pd.DataFrame({'spike_clusters': spike_clusters, 'spike_times': spike_times.T[0]})

        merged_df = spike_df.merge(cluster_info_df, left_on='spike_clusters', right_on='cluster_id', how="left")
        merged_df["recording_name"] = recording_basename

        merged_df["timestamp_isi"] = merged_df.groupby('spike_clusters')["spike_times"].diff()
        merged_df["current_isi"] = merged_df["timestamp_isi"] / SAMPLING_RATE
        
        if not merged_df.empty:
            recording_to_spike_df[recording_basename] = merged_df
       
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/cluster_info.tsv'


In [27]:
cluster_info_df.head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,2,0,0,0,24.692951,1,0.0,20.0,1.070175,good,3658,0,3.0
1,3,235,106,472,10.347874,1,0.0,20.0,14.493880,mua,49542,0,4.0
2,4,64,25,149,11.953332,4,0.0,80.0,27.163371,mua,92848,0,5.0
3,10,69,29,194,10.297276,18,0.0,360.0,6.455279,mua,22065,0,12.0
4,11,619,261,1295,9.638424,19,0.0,380.0,18.195901,mua,62196,0,13.0


In [28]:
spike_times

array([[     160],
       [     483],
       [     503],
       ...,
       [68361973],
       [68362204],
       [68362381]])

In [29]:
spike_clusters

array([ 4,  4, 11, ...,  4,  4, 87], dtype=int32)

- Combining the spike time df for all recordings

In [30]:
all_spike_time_df = pd.concat(recording_to_spike_df.values())

In [31]:
all_spike_time_df = all_spike_time_df[all_spike_time_df["group"] == "good"].reset_index(drop=True)

In [32]:
all_spike_time_df.head()

,spike_clusters,spike_times,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi
0,42,120,42,6,1,8,11.904501,20,0.0,400.0,12.966718,good,44322,0,44.0,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN
1,42,262,42,6,1,8,11.904501,20,0.0,400.0,12.966718,good,44322,0,44.0,20230620_114347_standard_comp_to_omission_D4_s...,142.0,0.00710
2,207,439,207,1,0,5,16.813625,17,0.0,340.0,4.308197,good,14726,0,36.0,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN
3,207,632,207,1,0,5,16.813625,17,0.0,340.0,4.308197,good,14726,0,36.0,20230620_114347_standard_comp_to_omission_D4_s...,193.0,0.00965
4,262,723,262,0,0,0,20.012720,19,0.0,380.0,2.037368,good,6964,0,39.0,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN


In [33]:
all_spike_time_df.tail()

,spike_clusters,spike_times,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi
2017369,64,68356117,64,4,1,12,10.794415,14,0.0,280.0,7.735803,good,26442,0,9.0,20230620_114347_standard_comp_to_omission_D4_s...,392.0,0.0196
2017370,89,68356258,89,11,11,13,17.238409,26,0.0,520.0,15.528948,good,53080,0,21.0,20230620_114347_standard_comp_to_omission_D4_s...,3360.0,0.1680
2017371,64,68357205,64,4,1,12,10.794415,14,0.0,280.0,7.735803,good,26442,0,9.0,20230620_114347_standard_comp_to_omission_D4_s...,1088.0,0.0544
2017372,89,68357578,89,11,11,13,17.238409,26,0.0,520.0,15.528948,good,53080,0,21.0,20230620_114347_standard_comp_to_omission_D4_s...,1320.0,0.0660
2017373,64,68358945,64,4,1,12,10.794415,14,0.0,280.0,7.735803,good,26442,0,9.0,20230620_114347_standard_comp_to_omission_D4_s...,1740.0,0.0870


## Grouping all the neurons by recording

In [47]:
# Grouping all spike times by neuron and recording
grouped_df = all_spike_time_df.groupby(['spike_clusters', 'recording_name'])["spike_times"].apply(lambda x: np.array(x)).reset_index()
grouped_df = grouped_df.sort_values(by=['recording_name', 'spike_clusters']).reset_index(drop=True)

In [48]:
grouped_df.head()

,spike_clusters,recording_name,spike_times
0,3,20230617_115521_standard_comp_to_omission_D1_s...,"[673223, 2367433, 2569212, 3969647, 3973243, 3..."
1,4,20230617_115521_standard_comp_to_omission_D1_s...,"[1071, 3665, 4592, 8167, 8358, 8807, 9043, 990..."
2,5,20230617_115521_standard_comp_to_omission_D1_s...,"[90780, 93969, 96069, 145651, 149204, 150749, ..."
3,6,20230617_115521_standard_comp_to_omission_D1_s...,"[102, 1962, 3893, 15127, 31864, 36704, 38461, ..."
4,17,20230617_115521_standard_comp_to_omission_D1_s...,"[151072, 201333, 206934, 213398, 220860, 23372..."


In [49]:
max_number_of_spikes = all_spike_time_df["n_spikes"].max()

In [50]:
max_number_of_spikes

179245

In [52]:
grouped_df["spike_times"] = grouped_df["spike_times"].apply(lambda x: np.concatenate([x, np.full([max_number_of_spikes - x.shape[0]], -1)]).astype(int))

In [53]:
grouped_df = grouped_df.groupby('recording_name').agg({'spike_clusters': lambda x: list(x), 'spike_times': lambda x: np.array(list(x))}).reset_index()

In [56]:
LFP_SPECTRAL_DF = pd.merge(LFP_SPECTRAL_DF, grouped_df, left_on='recording', right_on="recording_name", how='inner')

In [57]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,agent,subject_locations,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port,recording_name,spike_clusters,spike_times
0,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,1.1,...,None,"[[[40.039741738777444, 9.47029992556593], [42....",NaN,"[14.84, 13.97, 13.13, 12.32, 11.54, 10.78, 10....",NaN,"[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...",NaN,20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223, 2367433, 2569212, 3969647, 3973243, ..."
1,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,1.1,...,1.2,"[[[57.86163468579171, 31.554562356382696], [59...","[[[40.039741738777444, 9.47029992556593], [42....","[8.96, 8.195, 7.54, 7.004, 6.605, 6.348, 6.227...","[11.19, 10.55, 9.92, 9.31, 8.734, 8.18, 7.64, ...","[11.19, 11.234, 11.73, 12.34, 12.56, 13.58, 13...","[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...",20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223, 2367433, 2569212, 3969647, 3973243, ..."
2,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,1.2,...,1.1,"[[[40.039741738777444, 9.47029992556593], [42....","[[[57.86163468579171, 31.554562356382696], [59...","[11.19, 10.55, 9.92, 9.31, 8.734, 8.18, 7.64, ...","[8.96, 8.195, 7.54, 7.004, 6.605, 6.348, 6.227...","[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...","[11.19, 11.234, 11.73, 12.34, 12.56, 13.58, 13...",20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958, 5233, 6331, 7475, 8599, 10550, 23459, 2..."
3,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 5], [12, 13], [13...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,1.2,...,None,"[[[48.82535585060548, 37.83187483510619], [47....",NaN,"[5.184, 4.777, 4.383, 4.0, 3.629, 3.271, 2.926...",NaN,"[6.223, 6.227, 6.234, 6.24, 6.203, 6.2, 6.203,...",NaN,20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958, 5233, 6331, 7475, 8599, 10550, 23459, 2..."
4,2,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,1.1,...,1.4,"[[[46.838967596646405, 35.8797798363605], [44....","[[[31.37122566669405, 31.1085619584881], [33.5...","[1.143, 1.143, 1.142, 1.139, 1.135, 1.129, 1.1...","[0.6333, 0.5635, 0.4963, 0.4321, 0.3713, 0.313...","[5.72

In [60]:
example_arr = LFP_SPECTRAL_DF["spike_times"].iloc[0].copy()

In [62]:
LFP_SPECTRAL_DF["tone_timestamps"].iloc[0]

array([[ 1030879,  1230878],
       [ 3430878,  3630878],
       [ 5730880,  5930880],
       [ 7530877,  7730877],
       [ 8630880,  8830879],
       [ 9630879,  9830879],
       [11530876, 11730879],
       [13030876, 13230878],
       [15130878, 15330878],
       [16130878, 16330875],
       [17730875, 17930875],
       [19030875, 19230875],
       [20330877, 20530875],
       [21330874, 21530875],
       [22730874, 22930874],
       [24630874, 24830876],
       [25030476, 31380073],
       [32580474, 32780475],
       [34180472, 34380472],
       [35580474, 35780471],
       [37780473, 37980473],
       [38980473, 39180470],
       [44980471, 45180471],
       [46180471, 46380468],
       [47280468, 47480468],
       [48580470, 48780470],
       [49680475, 49880470],
       [50680470, 50880467],
       [51680470, 51880467],
       [53780466, 53980471],
       [56180466, 56380468],
       [58580465, 58780465],
       [60780465, 60980464],
       [62480469, 62680466],
       [648804

In [ ]:
raise ValueError()

## Calculating the firing rates

In [ ]:
LFP_SPECTRAL_DF.columns

In [ ]:
cluster_info_df

In [ ]:
LFP_SPECTRAL_DF.head()

In [ ]:
LFP_SPECTRAL_DF["neuron_average_fr"] = LFP_SPECTRAL_DF.apply(lambda x: np.array([neuron.spikes.calculate_rolling_avg_firing_rate(np.array(times), stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[0] for times in x["spike_times"]]), axis=1)

In [ ]:
LFP_SPECTRAL_DF["neuron_average_timestamps"] = LFP_SPECTRAL_DF.apply(lambda x: neuron.spikes.calculate_rolling_avg_firing_rate(x["spike_times"][0], stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[1], axis=1)

In [ ]:
LFP_SPECTRAL_DF["neuron_average_fr"] = LFP_SPECTRAL_DF.apply(lambda x: x["neuron_average_fr"] * SPIKE_WINDOW, axis=1)

In [ ]:
LFP_SPECTRAL_DF.iloc[:5,:10]

In [ ]:
LFP_SPECTRAL_DF.iloc[:5,10:20]

In [ ]:
LFP_SPECTRAL_DF.iloc[:5,20:30]

In [ ]:
LFP_SPECTRAL_DF.iloc[:5, 30:]

In [ ]:
LFP_SPECTRAL_DF["neuron_average_timestamps"].iloc[0].shape

In [ ]:
LFP_SPECTRAL_DF["neuron_average_fr"].iloc[0].shape

In [ ]:
np.max(LFP_SPECTRAL_DF["neuron_average_fr"].iloc[0])

In [ ]:
LFP_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError("")